In [25]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import panel as pn
from panel.interact import interact
from panel import widgets
pn.extension('plotly')
import hvplot.pandas
from pathlib import Path
import datetime as dt 
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv

In [26]:
load_dotenv()

True

In [27]:
# Import the CSVs RobinTrack (rt) to Pandas DataFrames
abb_path = Path("../data/cleandata/Robintrack_Data/ABB_csv_clean.csv")
amd_path = Path("../data/cleandata/Robintrack_Data/AMD_csv_clean.csv")
riot_path = Path("../data/cleandata/Robintrack_Data/RIOT_csv_clean.csv")
tsla_path = Path("../data/cleandata/Robintrack_Data/TSLA_csv_clean.csv")

abb_rt_data = pd.read_csv(abb_path, infer_datetime_format=True, parse_dates=True)
amd__rt_data = pd.read_csv(amd_path, infer_datetime_format=True, parse_dates=True)
riot_rt_data = pd.read_csv(riot_path, infer_datetime_format=True, parse_dates=True)
tsla_data = pd.read_csv(tsla_path, infer_datetime_format=True, parse_dates=True)

In [28]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("APCA_API_KEY")
alpaca_secret_key = os.getenv("APCA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [29]:
type(alpaca_api_key)

str

In [30]:
# Format current date as ISO format
start_date = pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()

end_date = pd.Timestamp("2020-08-10", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["ABB", "AMD", "RIOT", "TSLA"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for "ABB", "AMD", "RIOT", and "TSLA"
shares_df = api.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

close_price_stocks= shares_df.xs("close", axis='columns', level=1)

In [31]:
# Define Panel Visualization Functions
def robintrack_ABB_stock_price ():
    """Robintrack AAB's accounts effect on ABB's price."""
    abb_rt_data = abb_rt_data.resample('D', on='timestamp').mean()
    abb_rt_data['users_holding'] = abb_rt_data['users_holding'].fillna(2147.00)
    close_price_stocks["ABB"] = close_price_stocks["ABB"]["close"]
    close_price_stocks.index = close_price_stocks.index.date
    concat_robin_ticker = pd.concat([close_price_stocks, abb_rt_data], axis="columns", join="inner")
    daily_ret_abb = concat_robin_ticker.pct_change()
    abb_corr_plot = daily_ret_abb.hvplot.line(title='RobinTrack ABB Accounts Effect in ABB Stock Price')
    return abb_corr_plot
    

    

# Panel Dashboard

In [32]:
abb_analysis= pn.Row("## ABB RobinTrack and Stock Price Analysis", robintrack_ABB_stock_price ())

# Create tabs

robintrack_stock_analysis_dashboard = pn.Tabs("ABB Analysis", abb_analysis)

dashboard= pn.Column(pn.Row("Effect of Number of Individual Robinhood on Unerlying Price & Option Volatility for RIOT, AMD, ABB, and TSLA"), 
                     robintrack_stock_analysis_dashboard)

UnboundLocalError: local variable 'abb_rt_data' referenced before assignment

# Serve the Panel Dashboard

In [ ]:
dashboard.servable()